In [177]:
### Librerías
import pandas as pd
import numpy as np
import yfinance as yf
from alphacast import Alphacast
import datetime as dt
from sqlalchemy import create_engine
import pyodbc


In [178]:
### CONEXIÓN A SQL SERVER
from sqlalchemy import create_engine

# Configuración de la conexión a SQL Server
usuario = 'sa'   # Usa tu usuario de SQL Server (si aplica)
contraseña = '1234' # Contraseña del usuario de SQL Server (si aplica)
host = 'DESKTOP-SJBS2KI\SQLEXPRESS2'  # Nombre de tu servidor SQL
base_de_datos = 'Proyecto_BI_Finance' # Nombre de tu base de datos

# 💡 Cadena de conexión usando SQL Server con ODBC Driver 17
cadena_conexion = f"mssql+pyodbc://{usuario}:{contraseña}@{host}/{base_de_datos}?driver=ODBC+Driver+17+for+SQL+Server"

# Crea el motor de conexión
motor_sql_server = create_engine(cadena_conexion)

# 🔹 Probar la conexión
try:
    conexion = motor_sql_server.connect()
    print("✅ Conexión exitosa a SQL Server")
    conexion.close()
except Exception as e:
    print("❌ Error de conexión:", e)


✅ Conexión exitosa a SQL Server


In [179]:
### CONEXIÓN A BBDD MYSQL

# Configura la conexión a la base de datos
usuario = 'root'
contraseña = 'root'
host = 'localhost'
base_de_datos = 'proyecto_bi_finance'

# Crea la cadena de conexión
cadena_conexion = f'mysql+pymysql://{usuario}:{contraseña}@{host}/{base_de_datos}'

# Crea el motor de SQLAlchemy
motor_mysql = create_engine(cadena_conexion)

# DATOS HISTORICOS 

### ACCIONES PANEL LIDER

In [180]:
# Listado de tickers Panel Líder
tickers = [
    "ALUA.BA", "VALO.BA", "BBAR.BA", "BMA.BA", "SUPV.BA", "BYMA.BA", "CVH.BA", "CEPU.BA", 
    "COME.BA", "CRES.BA", "EDN.BA", "GGAL.BA", "MIRG.BA", "PAMP.BA", "TECO2.BA", "TXAR.BA", 
    "TGSU2.BA", "TRAN.BA", "TGNO4.BA", "YPFD.BA"
]

# Crear un DataFrame vacío para almacenar los datos históricos
AccionesLideres = pd.DataFrame()

# Obtener datos históricos para cada ticker
for ticker in tickers:
    stock = yf.Ticker(ticker)
    hist = stock.history(start="2023-01-01") 
    hist['Ticker'] = ticker  # Agregar el ticker como una columna
    AccionesLideres = pd.concat([AccionesLideres, hist])

AccionesLideres = AccionesLideres[['Ticker', 'Close']]
AccionesLideres.reset_index(inplace=True)
AccionesLideres['Date'] = AccionesLideres['Date'].dt.strftime('%Y-%m-%d')

AccionesLideres[['Tipo_activo', 'Sector']] = ["Merval", "N/A"]

AccionesLideres.head(10)


,Date,Ticker,Close,Tipo_activo,Sector
0,2023-01-02,ALUA.BA,181.714798,Merval,N/A
1,2023-01-03,ALUA.BA,180.493622,Merval,N/A
2,2023-01-04,ALUA.BA,182.203278,Merval,N/A
3,2023-01-05,ALUA.BA,185.622650,Merval,N/A
4,2023-01-06,ALUA.BA,189.530487,Merval,N/A
5,2023-01-09,ALUA.BA,193.194107,Merval,N/A
6,2023-01-10,ALUA.BA,197.346191,Merval,N/A
7,2023-01-11,ALUA.BA,196.613464,Merval,N/A
8,2023-01-12,ALUA.BA,202.231003,Merval,N/A
9,2023-01-13,ALUA.BA,216.885422,Merval,N/A


### BONO AL30

In [181]:
# Clave de API de Alphacast
API_KEY = 'ak_fd398JRrB0Rd9TmmjKsO'

# Inicializar la conexión con Alphacast
alphacast = Alphacast(API_KEY)

# Utilizamos el dataset de Bonos soberanos (ID 5359)
dataset = alphacast.datasets.dataset(5359)

# Definir las variables (columnas) que queremos filtrar
variables_deseadas = ["Fecha", "AL30C"]

# Descargar los datos en formato pandas, filtrando las variables
Bonos = dataset.download_data(
    format="pandas", 
    startDate=dt.date(2023, 1, 1),  # Fecha de inicio: hace 1 año
    endDate=None,                                          # Fecha de fin: hoy (por defecto)
    filterVariables=variables_deseadas,                    # Filtrar solo las variables deseadas
    filterEntities={}                                      # No filtrar entidades
)

Bonos.rename(columns={"AL30C": "Close"}, inplace=True)
Bonos['Ticker'] = 'AL30C'
Bonos = Bonos[['Date', 'Close','Ticker']]
Bonos[['Tipo_activo', 'Sector']]  = ["Bonos","N/A"]
Bonos.head(10)

,Date,Close,Ticker,Tipo_activo,Sector
0,2023-01-02,NaN,AL30C,Bonos,N/A
1,2023-01-03,24.5,AL30C,Bonos,N/A
2,2023-01-04,24.0,AL30C,Bonos,N/A
3,2023-01-05,25.7,AL30C,Bonos,N/A
4,2023-01-06,26.7,AL30C,Bonos,N/A
5,2023-01-09,25.7,AL30C,Bonos,N/A
6,2023-01-10,26.0,AL30C,Bonos,N/A
7,2023-01-11,27.7,AL30C,Bonos,N/A
8,2023-01-12,27.9,AL30C,Bonos,N/A
9,2023-01-13,27.9,AL30C,Bonos,N/A


### FCI

In [182]:
"""

# Clave de API de Alphacast
API_KEY = 'ak_fd398JRrB0Rd9TmmjKsO'

# Inicializar la conexión con Alphacast
alphacast = Alphacast(API_KEY)

# Utilizamos el dataset de Bonos soberanos (ID 29806)
dataset = alphacast.datasets.dataset(29806)

# Descargar los datos en formato pandas
FCIs = dataset.download_data(
    format="pandas", 
    startDate=dt.date(2023, 1, 1),  # Fecha de inicio: 01/01/2023
    endDate=dt.date.today(),        # Fecha de fin: hoy
    filterVariables={},             # No filtrar variables específicas
    filterEntities={}               # No filtrar entidades
)

# Renombrar la columna "Fecha" a "Date" para mantener consistencia
FCIs.rename(columns={"Fecha": "Date", "nombre":"Ticker","Patrimonio Neto": "Close"}, inplace=True)

# Filtrar solo los instrumentos deseados en la columna "Nombre"
instrumentos_deseados = ["Fima Renta Pesos", "Allaria Dolares Plus", "Fima Acciones"]
FCIs = FCIs[FCIs["Ticker"].isin(instrumentos_deseados)]


FCIs = FCIs[['Date', 'Ticker', 'Close']]

FCIs[['Tipo_activo', 'Sector']] = ["FCI","N/A"]
# Mostrar las primeras 10 filas del DataFrame filtrado
FCIs.head(10)

"""

'\n\n# Clave de API de Alphacast\nAPI_KEY = \'ak_fd398JRrB0Rd9TmmjKsO\'\n\n# Inicializar la conexión con Alphacast\nalphacast = Alphacast(API_KEY)\n\n# Utilizamos el dataset de Bonos soberanos (ID 29806)\ndataset = alphacast.datasets.dataset(29806)\n\n# Descargar los datos en formato pandas\nFCIs = dataset.download_data(\n    format="pandas", \n    startDate=dt.date(2023, 1, 1),  # Fecha de inicio: 01/01/2023\n    endDate=dt.date.today(),        # Fecha de fin: hoy\n    filterVariables={},             # No filtrar variables específicas\n    filterEntities={}               # No filtrar entidades\n)\n\n# Renombrar la columna "Fecha" a "Date" para mantener consistencia\nFCIs.rename(columns={"Fecha": "Date", "nombre":"Ticker","Patrimonio Neto": "Close"}, inplace=True)\n\n# Filtrar solo los instrumentos deseados en la columna "Nombre"\ninstrumentos_deseados = ["Fima Renta Pesos", "Allaria Dolares Plus", "Fima Acciones"]\nFCIs = FCIs[FCIs["Ticker"].isin(instrumentos_deseados)]\n\n\nFCIs = FC

### STOCKS

In [183]:
# Lista de tickers
tickers = ["XOM", "CVX", "NEE", "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", 
           "WMT", "PG", "KO", "PEP", "JNJ", "PFE", "MRK", "UNH", 
           "JPM", "BAC", "V", "MA", "BA", "MMM", "VZ", "T", "DOW"]

# Crear un DataFrame vacío para almacenar todos los datos
Stocks = pd.DataFrame()

# Obtener datos históricos para cada ticker
for ticker in tickers:
    stock = yf.Ticker(ticker)
    hist = stock.history(start="2023-01-01") 
    hist['Ticker'] = ticker  # Agregar el ticker como una columna
    Stocks = pd.concat([Stocks, hist])

Stocks = Stocks[['Ticker', 'Close']]
Stocks.reset_index(inplace=True)
Stocks['Date'] = Stocks['Date'].dt.strftime('%Y-%m-%d')

Stocks['Tipo_activo'] ="Wall Street"
Stocks.head(10)

,Date,Ticker,Close,Tipo_activo
0,2023-01-03,XOM,98.713249,Wall Street
1,2023-01-04,XOM,99.000565,Wall Street
2,2023-01-05,XOM,101.215607,Wall Street
3,2023-01-06,XOM,102.438965,Wall Street
4,2023-01-09,XOM,100.529778,Wall Street
5,2023-01-10,XOM,102.031197,Wall Street
6,2023-01-11,XOM,103.217499,Wall Street
7,2023-01-12,XOM,104.932053,Wall Street
8,2023-01-13,XOM,104.867195,Wall Street
9,2023-01-17,XOM,104.663300,Wall Street


In [184]:
# Lista de tickers y sectores
tickers_sectores = {
    "XOM": "Energía",
    "CVX": "Energía",
    "NEE": "Energía",
    "AAPL": "Tecnología",
    "MSFT": "Tecnología",
    "GOOGL": "Tecnología",
    "AMZN": "Tecnología",
    "NVDA": "Tecnología",
    "WMT": "Consumo Masivo",
    "PG": "Consumo Masivo",
    "KO": "Consumo Masivo",
    "PEP": "Consumo Masivo",
    "JNJ": "Salud",
    "PFE": "Salud",
    "MRK": "Salud",
    "UNH": "Salud",
    "JPM": "Finanzas",
    "BAC": "Finanzas",
    "V": "Finanzas",
    "MA": "Finanzas",
    "BA": "Industrial",
    "MMM": "Industrial",
    "DOW": "Industrial"
}

# Agregar la columna "Sector" usando el mapeo
Stocks['Sector'] = Stocks['Ticker'].map(tickers_sectores)

# Mostrar las primeras filas del DataFrame
Stocks.head(10)

,Date,Ticker,Close,Tipo_activo,Sector
0,2023-01-03,XOM,98.713249,Wall Street,Energía
1,2023-01-04,XOM,99.000565,Wall Street,Energía
2,2023-01-05,XOM,101.215607,Wall Street,Energía
3,2023-01-06,XOM,102.438965,Wall Street,Energía
4,2023-01-09,XOM,100.529778,Wall Street,Energía
5,2023-01-10,XOM,102.031197,Wall Street,Energía
6,2023-01-11,XOM,103.217499,Wall Street,Energía
7,2023-01-12,XOM,104.932053,Wall Street,Energía
8,2023-01-13,XOM,104.867195,Wall Street,Energía
9,2023-01-17,XOM,104.663300,Wall Street,Energía


### CRIPTO

In [185]:
# Clave de API de Alphacast
API_KEY = 'ak_fd398JRrB0Rd9TmmjKsO'

# Inicializar la conexión con Alphacast
alphacast = Alphacast(API_KEY)

# Utilizamos el dataset de Cripto soberanos (ID 7731)
dataset = alphacast.datasets.dataset(7731)

# Descargar los datos en formato pandas, filtrando las variables
Cripto = dataset.download_data(
    format="pandas", 
    startDate=dt.date(2023, 1, 1),  # Fecha de inicio
    endDate=None,                                          # Fecha de fin: hoy (por defecto)
    filterVariables={},                    # Filtrar solo las variables deseadas
    filterEntities={}                                      # No filtrar entidades
)

# Filtrar solo las cripto deseados en la columna "CriptoAsset"
criptos_top5 = ['ADA-USD','BTC-USD','BNB-USD','SOL-USD','ETH-USD']
Cripto = Cripto[Cripto["CriptoAsset"].isin(criptos_top5)]

Cripto = Cripto[['Date','CriptoAsset', 'Close']]
Cripto.rename(columns={"CriptoAsset": "Ticker"}, inplace=True)

Cripto[['Tipo_activo', 'Sector']]= ["Cripto","N/A"]
Cripto.head(10)


,Date,Ticker,Close,Tipo_activo,Sector
3,2023-01-01,ADA-USD,0.249771,Cripto,N/A
26,2023-01-01,BNB-USD,244.136978,Cripto,N/A
29,2023-01-01,BTC-USD,16625.080078,Cripto,N/A
60,2023-01-01,ETH-USD,1200.964844,Cripto,N/A
183,2023-01-02,ADA-USD,0.253828,Cripto,N/A
206,2023-01-02,BNB-USD,245.535904,Cripto,N/A
209,2023-01-02,BTC-USD,16688.470703,Cripto,N/A
240,2023-01-02,ETH-USD,1214.656616,Cripto,N/A
363,2023-01-03,ADA-USD,0.252796,Cripto,N/A
386,2023-01-03,BNB-USD,246.133362,Cripto,N/A


### FX

In [186]:
# Clave de API de Alphacast
API_KEY = 'ak_fd398JRrB0Rd9TmmjKsO'

# Inicializar la conexión con Alphacast
alphacast = Alphacast(API_KEY)

# Utilizamos el dataset de FX_tmp soberanos (ID 7731)
dataset = alphacast.datasets.dataset(5288)

# Descargar los datos en formato pandas, filtrando las variables
FX_tmp = dataset.download_data(
    format="pandas", 
    startDate=dt.date(2023, 1, 1),  # Fecha de inicio
    endDate=None,                                          # Fecha de fin: hoy (por defecto)
    filterVariables={},                    # Filtrar solo las variables deseadas
    filterEntities={}                                      # No filtrar entidades
)


FX_tmp = FX_tmp[['Date','Dolar MEP', 'Dolar Oficial']]

# Derretir el DataFrame
FX = pd.melt(
    FX_tmp,
    id_vars=['Date'],          # Columna que se mantiene fija
    value_vars=['Dolar MEP', 'Dolar Oficial'],  # Columnas a unificar
    var_name='Ticker',        # Nombre de la nueva columna que contendrá 'Dolar MEP' y 'Dolar Oficial'
    value_name='Precio'       # Nombre de la nueva columna que contendrá los valores
)

# Ordenar el DataFrame por fecha (opcional)
FX = FX.sort_values(by='Date').reset_index(drop=True)
FX.rename(columns={"Precio":"Close"}, inplace=True)

FX[['Tipo_activo', 'Sector']]= ["FX","N/A"]
# Mostrar el resultado
FX.head()

,Date,Ticker,Close,Tipo_activo,Sector
0,2023-01-02,Dolar MEP,329.21,FX,N/A
1,2023-01-02,Dolar Oficial,185.36,FX,N/A
2,2023-01-03,Dolar Oficial,185.99,FX,N/A
3,2023-01-03,Dolar MEP,330.70,FX,N/A
4,2023-01-04,Dolar MEP,334.93,FX,N/A


### DATASET FINAL

In [187]:
# Lista de DataFrames
dataframes = [AccionesLideres, Bonos, Cripto, Stocks, FX]

# Concatenar todos los DataFrames en uno solo
df_datos_bursatiles = pd.concat(dataframes, ignore_index=True)

df_datos_bursatiles.head(15)

,Date,Ticker,Close,Tipo_activo,Sector
0,2023-01-02,ALUA.BA,181.714798,Merval,N/A
1,2023-01-03,ALUA.BA,180.493622,Merval,N/A
2,2023-01-04,ALUA.BA,182.203278,Merval,N/A
3,2023-01-05,ALUA.BA,185.622650,Merval,N/A
4,2023-01-06,ALUA.BA,189.530487,Merval,N/A
5,2023-01-09,ALUA.BA,193.194107,Merval,N/A
6,2023-01-10,ALUA.BA,197.346191,Merval,N/A
7,2023-01-11,ALUA.BA,196.613464,Merval,N/A
8,2023-01-12,ALUA.BA,202.231003,Merval,N/A
9,2023-01-13,ALUA.BA,216.885422,Merval,N/A


#### ANALISIS DE NULOS

In [188]:
# Verificar valores nulos por columna
nulos_por_columna = df_datos_bursatiles.isnull().sum()

# Mostrar el resultado
print(nulos_por_columna)

Date              0
Ticker            0
Close             9
Tipo_activo       0
Sector         1122
dtype: int64


In [189]:
# Dropear Nulos
df_datos_bursatiles.dropna(inplace=True)

In [190]:
# Obtener la fecha máxima para cada Ticker
fecha_maxima_por_ticker = df_datos_bursatiles.groupby('Ticker')['Date'].max().reset_index()

# Mostrar el resultado
print(fecha_maxima_por_ticker)

           Ticker        Date
0            AAPL  2025-03-28
1         ADA-USD  2025-03-28
2           AL30C  2025-03-28
3         ALUA.BA  2025-03-28
4            AMZN  2025-03-28
5              BA  2025-03-28
6             BAC  2025-03-28
7         BBAR.BA  2025-03-28
8          BMA.BA  2025-03-28
9         BNB-USD  2025-03-28
10        BTC-USD  2025-03-28
11        BYMA.BA  2025-03-28
12        CEPU.BA  2025-03-28
13        COME.BA  2025-03-28
14        CRES.BA  2025-03-28
15         CVH.BA  2025-03-28
16            CVX  2025-03-28
17            DOW  2025-03-28
18      Dolar MEP  2025-03-27
19  Dolar Oficial  2025-03-27
20         EDN.BA  2025-03-28
21        ETH-USD  2025-03-28
22        GGAL.BA  2025-03-28
23          GOOGL  2025-03-28
24            JNJ  2025-03-28
25            JPM  2025-03-28
26             KO  2025-03-28
27             MA  2025-03-28
28        MIRG.BA  2025-03-28
29            MMM  2025-03-28
30            MRK  2025-03-28
31           MSFT  2025-03-28
32        

In [191]:
# Concatenamos las columnas 'Tipo_activo' y 'Sector' con un guion (-) en medio
# Si 'Sector' es 'N/A', dejamos 'Tipo_activo' como está, sino concatenamos 'Tipo_activo' con 'Sector'
df_datos_bursatiles['Tipo_activo'] = np.where(df_datos_bursatiles['Sector'] == 'N/A', 
                                               df_datos_bursatiles['Tipo_activo'], 
                                               df_datos_bursatiles['Tipo_activo'] + ' - ' + df_datos_bursatiles['Sector'])



In [192]:
df_datos_bursatiles.Tipo_activo.unique()

array(['Merval', 'Bonos', 'Cripto', 'Wall Street - Energía',
       'Wall Street - Tecnología', 'Wall Street - Consumo Masivo',
       'Wall Street - Salud', 'Wall Street - Finanzas',
       'Wall Street - Industrial', 'FX'], dtype=object)

# CARTERAS DE INVERSION

### DESCRIPCIONES DE CARTERA

In [193]:
# Crear un DataFrame con los datos de las carteras
data = {
    'Cartera': [
        'Moderado',
        'Agresivo',
        'Conservador',
        'Full-Tech',
        'Made in USA'
    ],
    'Descripcion': [
        'Diseñada para inversores que buscan un equilibrio entre riesgo y rendimiento. Incluye una mezcla de acciones de crecimiento en USA y MERVAL, junto con bonos de calidad. Ideal para quienes tienen un horizonte de inversión a mediano plazo.',
        'Para inversores con alta tolerancia al riesgo y un horizonte de inversión a largo plazo. Esta cartera se centra en acciones de alto crecimiento, tecnología y criptomonedas, con el potencial de obtener rendimientos significativos.',
        'Ideal para inversores que priorizan la preservación del capital y buscan un bajo nivel de riesgo. Esta cartera se compone principalmente de bonos, fondos de renta fija y acciones de dividendos estables.',
        'Una cartera enfocada en el sector tecnológico, ideal para inversores que creen en el crecimiento continuo de la innovación. Incluye acciones de empresas líderes en tecnología, software, cripto y startups disruptivas.',
        'Esta cartera apuesta a la reactivación económica y comercial de Estados Unidos, con un enfoque en acciones de consumo masivo e industria norteamericana. Perfecta para inversores que confían en el crecimiento interno de EE. UU.'
    ]
}

# Crear el DataFrame
df_carteras = pd.DataFrame(data)

# Crear el DataFrame con un índice que empiece en 1
df_carteras = pd.DataFrame(data, index=range(1, len(data['Cartera']) + 1))
# Convertir el índice en una columna llamada 'id'
df_carteras = df_carteras.reset_index().rename(columns={'index': 'Id'})
df_carteras.head()

,Id,Cartera,Descripcion
0,1,Moderado,Diseñada para inversores que buscan un equilib...
1,2,Agresivo,Para inversores con alta tolerancia al riesgo ...
2,3,Conservador,Ideal para inversores que priorizan la preserv...
3,4,Full-Tech,"Una cartera enfocada en el sector tecnológico,..."
4,5,Made in USA,Esta cartera apuesta a la reactivación económi...


### COMPOSICIÓN DE CARTERA

In [194]:
# Definir las carteras y sus composicion_carteras (ajustadas según tus comentarios)
data = {
    #Moderado
    "1": {
        "Merval": 15,
        "Wall Street - Consumo Masivo" : 25,
        "Wall Street - Finanzas" : 10,
        "Wall Street - Energía": 10,
        "Bonos": 10,
        "Cripto": 10,  # Un poco de cripto
        "FX": 20
    },
    #Agresivo
    "2": {
        "Merval": 20,  # Menos rval
        "Wall Street - Tecnología" : 10,
        "Wall Street - Finanzas" : 10,
        "Wall Street - Energía": 10,
        "Wall Street - Salud": 10,
        "Bonos": 10,
        "Cripto": 20,  # Más cripto
        "FX": 10
    },
    #Conservador
    "3": {
        "Merval": 5,
        "Wall Street - Energía": 10,
        "Wall Street - Industrial": 35,  # Más Wall Street
        "Wall Street - Consumo Masivo": 30,
        "Bonos": 10,
        "Cripto": 0,  # 0 cripto
        "FX": 10
    },
    #Full-Tech
    "4": {
        "Merval": 0,
        "Wall Street - Tecnología": 70,
        "Bonos": 0,
        "Cripto": 30,
        "FX": 0
    },
    #Made in USA Revival
    "5": {
        "Merval": 0,
        "Wall Street - Industrial": 50,  # Todo industria y consumo
        "Wall Street - Consumo Masivo": 50,
        "Bonos": 0,  # 0 bonos
        "Cripto": 0,
        "FX": 0
    }
}

# Crear el DataFrame
composicion_carteras = pd.DataFrame(data)

# Reemplazar NaN con 0 (por si alguna cartera no tiene un instrumento)
composicion_carteras = composicion_carteras.fillna(0)

# Pivotar el DataFrame
composicion_carteras_final = composicion_carteras.reset_index().melt(
    id_vars='index',  # Mantener la columna de instrumentos
    var_name='Id Cartera',  # Nombre de la columna para las carteras
    value_name='%'  # Nombre de la columna para los porcentajes
)

# Renombrar columnas
composicion_carteras_final = composicion_carteras_final.rename(columns={'index': 'Tipo_activo'})

# Mostrar el DataFrame pivotado
composicion_carteras_final.head(30)

,Tipo_activo,Id Cartera,%
0,Merval,1,15.0
1,Wall Street - Consumo Masivo,1,25.0
2,Wall Street - Finanzas,1,10.0
3,Wall Street - Energía,1,10.0
4,Bonos,1,10.0
5,Cripto,1,10.0
6,FX,1,20.0
7,Wall Street - Tecnología,1,0.0
8,Wall Street - Salud,1,0.0
9,Wall Street - Industrial,1,0.0


#### ENVIO A DDBB

In [195]:
# Enviar los DataFrame a la base de datos
 
nombre_tabla = 'carteras_desc'

df_carteras.to_sql(nombre_tabla, con=motor_sql_server, if_exists='replace', index=False)

##########

nombre_tabla = 'carteras_comp'

composicion_carteras_final.to_sql(nombre_tabla, con=motor_sql_server, if_exists='replace', index=False)

#################
 
nombre_tabla = 'datos_historicos'


df_datos_bursatiles.to_sql(nombre_tabla, con=motor_sql_server, if_exists='replace', index=False)

277

### STORE CSV


In [196]:
# Guardar los DataFrame finales en un archivo CSV
df_datos_bursatiles.to_csv("datos_bursatiles_final.csv", index=False)
df_carteras.to_csv("carteras_desc.csv", index = False)
composicion_carteras_final.to_csv("carteras_comp.csv", index = False)